In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import GPTNeoForCausalLM, AutoTokenizer
from transformers import RobertaConfig
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
import torch
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Varun53/openai-roberta-large-AI-detection")

model = AutoModelForSequenceClassification.from_pretrained("Varun53/openai-roberta-large-AI-detection")

In [ ]:
text=str(input("""Enter the text: """))

Enter the text: Artificial Intelligence: The Next Frontier of Human Innovation  Artificial Intelligence (AI) represents one of the most transformative technologies of our time, reshaping industries, enhancing daily life, and challenging our understanding of intelligence and autonomy. Its profound impact stretches across diverse fields, promising unprecedented advancements and posing significant ethical and practical challenges. This essay explores the multifaceted nature of AI, its potential benefits, and the critical considerations it raises for the future.


In [ ]:
text

'Artificial Intelligence: The Next Frontier of Human Innovation  Artificial Intelligence (AI) represents one of the most transformative technologies of our time, reshaping industries, enhancing daily life, and challenging our understanding of intelligence and autonomy. Its profound impact stretches across diverse fields, promising unprecedented advancements and posing significant ethical and practical challenges. This essay explores the multifaceted nature of AI, its potential benefits, and the critical considerations it raises for the future.'

In [ ]:
inputs=tokenizer(text,padding=True, truncation=True, return_tensors="pt")

In [ ]:
def text_to_sentences(text):
    clean_text = text.replace('\n', ' ')
    return re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', clean_text)


def chunks_of_2000(text, chunk_size = 2000):
    sentences = text_to_sentences(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk + sentence) <= chunk_size:
            if len(current_chunk)!=0:
                current_chunk += " "+sentence
            else:
                current_chunk += sentence
        else:
            chunks.append(current_chunk)
            current_chunk = sentence
    chunks.append(current_chunk)
    return chunks

def predict(query):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.model_max_length - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
      logits = model(**inputs).logits

      probs=logits.softmax(dim=-1)


    Ai_generated, Human_Generated = probs.detach().cpu().flatten().numpy().tolist()
    return Ai_generated

def findRealProb(text):
    chunksOfText = (chunks_of_2000(text))
    results = []
    for chunk in chunksOfText:
        output = predict(chunk)
        results.append([output, len(chunk)])

    ans = 0
    cnt = 0
    for prob, length in results:
        cnt += length
        ans = ans + prob*length
    realProb = ans/cnt
    return {"Ai_Generated": realProb, "Human_Generated": 1-realProb}, results

In [ ]:
findRealProb(text)

({'Ai_Generated': 0.6782939434051514, 'Human_Generated': 0.32170605659484863},
 [[0.6782939434051514, 548]])

In [ ]:
def get_ai_percentage(text):
    inputs=tokenizer(text,padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        ai_probs = logits.softmax(dim=-1)
    ai_percentage = ai_probs[0][0] / len(ai_probs)
    return ai_percentage * 100

In [ ]:
get_ai_percentage(text)

tensor(67.8294)

# **AI Text Humanizer with Synonym Replacement**


In [ ]:
!pip install spacy nltk transformers torch sentencepiece

In [ ]:
# Import dependencies
from transformers import AutoTokenizer, AutoModelForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration
import torch
import nltk
import spacy
from nltk.corpus import wordnet
import subprocess

# Download NLTK data (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')  # Download WordNet

# Download spaCy model if not already installed
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load("en_core_web_sm")

# Check for GPU and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load AI Detector model and tokenizer from Hugging Face (DistilBERT)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english").to(device)

# Load SRDdev Paraphrase model and tokenizer for humanizing text
paraphrase_tokenizer = T5Tokenizer.from_pretrained("SRDdev/Paraphrase")
paraphrase_model = T5ForConditionalGeneration.from_pretrained("SRDdev/Paraphrase").to(device)

# Function to find synonyms using WordNet via NLTK
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

# Replace words with synonyms using spaCy and WordNet
def replace_with_synonyms(text):
    doc = nlp(text)
    processed_text = []
    for token in doc:
        synonyms = get_synonyms(token.text.lower())
        if synonyms and token.pos_ in {"NOUN", "VERB", "ADJ", "ADV"}:  # Only replace certain types of words
            replacement = synonyms[0]  # Replace with the first synonym
            if token.is_title:
                replacement = replacement.capitalize()
            processed_text.append(replacement)
        else:
            processed_text.append(token.text)
    return " ".join(processed_text)

# AI detection function using DistilBERT
def detect_ai_generated(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = torch.softmax(outputs.logits, dim=1)
    ai_probability = probabilities[0][1].item()  # Probability of being AI-generated
    return ai_probability

# Humanize the AI-detected text using the SRDdev Paraphrase model
def humanize_text(AI_text):
    paragraphs = AI_text.split("\n")
    paraphrased_paragraphs = []
    for paragraph in paragraphs:
        if paragraph.strip():
            inputs = paraphrase_tokenizer(paragraph, return_tensors="pt", max_length=512, truncation=True).to(device)
            paraphrased_ids = paraphrase_model.generate(
                inputs['input_ids'],
                max_length=inputs['input_ids'].shape[-1] + 20,  # Slightly more than the original input length
                num_beams=4,
                early_stopping=True,
                length_penalty=1.0,
                no_repeat_ngram_size=3,
            )
            paraphrased_text = paraphrase_tokenizer.decode(paraphrased_ids[0], skip_special_tokens=True)
            paraphrased_paragraphs.append(paraphrased_text)
    return "\n\n".join(paraphrased_paragraphs)

# Main function to handle the overall process
def main_function(AI_text):
    # Replace words with synonyms
    text_with_synonyms = replace_with_synonyms(AI_text)

    # Detect AI-generated content
    ai_probability = detect_ai_generated(text_with_synonyms)

    # Humanize AI text
    humanized_text = humanize_text(text_with_synonyms)
    print(f"AI-Generated Content: {ai_probability:.2f}%")
    print("\nHumanized Text:")
    print(humanized_text)


In [ ]:
#"Enter AI-generated text and get a human-written version, with synonyms replaced for more natural output. This space uses models from Hugging Face directly."
AI_text=str(input("""Enter the text: """))
main_function(AI_text)

Enter the text: Artificial Intelligence: The Next Frontier of Human Innovation.  Artificial Intelligence (AI ) is a virtually transformative applied_science of our prison system, changing the way we manufacture, reshape our daily life, challenge our belief in intelligence information and self-sufficiency. Its profound impact stretches across a variety of domains, calls for an unprecedented procession and impersonates significant honorable and practical gainsay. This experiment examines the multifarious nature of AI, its likely benefit, and the vital consideration it has for future.
AI-Generated Content: 0.77%

Humanized Text:
AI is a revolutionary science that alters the organization of our prisons, shifts the way we work, recasts our daily lives, disputes our perception of intelligence data and ego-sufficiency. Its unrefined touch of intelligence and phone call for an unprecedented emanation and represents a new level of ethical and pragmatic uncertainty. This study examines the multi

'Done'